DeepBlink is a 2D CNN to detect spots:  

the network only works on 2d images but they support 3d by offering a non maximal suppression:  
https://github.com/BBQuercus/deepBlink/blob/master/examples/3d_prediction.ipynb  
(used below)

#### Installation

```
conda create -n deepblink python=3.7  
conda activate deepblink

## if GPUs are available install cuda and tf2_gpu according to the nvidia drive

conda install -c bbquercus deepblink

```

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
tf.config.experimental.list_physical_devices()

In [ ]:
is_gpu = len(tf.config.experimental.list_physical_devices('GPU')) > 0
is_gpu

In [ ]:
from glob import glob
import os
import pandas as pd
import tifffile as tif
import numpy as np
import time

import deepblink as pink
from skimage import io
import trackpy as tp
import subprocess

### Load pretrained model
We trained our own network, and results work well on the simulated data.  

In [ ]:
model = pink.io.load_model("230903_210048_deepBlink_is_sweet.h5")

### Load Images

In [ ]:
org_ims_dir = 'ufish/data_3d/train/image'
org_ims_paths = glob(os.path.join(org_ims_dir, '*.tif'))
len(org_ims_paths)

In [ ]:
results_path = "ufish/benchmarks/3d/DB/results"

## Predict 

#### In 2D

In [ ]:
## Grid search on this variable:
radius_refinement = [2,3,4]

In [ ]:
results_path_2d = os.path.join(results_path, '2D')
os.makedirs(results_path_2d, exist_ok=True)

In [ ]:
for i,im_path in enumerate(org_ims_paths):

    im_dir = os.path.basename(os.path.dirname(im_path))
    im_name = os.path.basename(im_path)

    im_3d = tif.imread(im_path)

    for j,r in enumerate(radius_refinement):

        pad_width = r
        
        df = pd.DataFrame()

        for slice, image_curr in enumerate(im_3d):

            # deepBlink prediction
            yx = pink.inference.predict(image=image_curr, model=model)
            y, x = yx.T.copy()

            # pad to avoid error for spot close to the edges
            yx = yx + pad_width
            image_curr = np.pad(
                image_curr, pad_width=pad_width, mode='constant', constant_values = 0
            )

            # Refinement with trackpy
            df_curr = tp.refine_com(
                raw_image=image_curr, image=image_curr, radius=r, coords=yx
            )
            df_curr["x"] = x
            df_curr["y"] = y
            df_curr["z"] = slice
            df = pd.concat([df, df_curr], ignore_index=True)

        df.to_csv(os.path.join(results_path_2d ,f'radius{r}__{im_dir}__{im_name}.csv'), index=False)


## Unite spots (2D to 3D)

In [ ]:
## grid search search_range)
search_range = [1,2,3]
gap_frames = 0
min_frames = 2

In [ ]:
results_path_3d = os.path.join(results_path, '3D')
os.makedirs(results_path_3d, exist_ok=True)

In [ ]:
times = [[[] for j in range(len(search_range))] for i in range(len(radius_refinement))] 

for i,im_path in enumerate(org_ims_paths):
    
    im_dir = os.path.basename(os.path.dirname(im_path))
    im_name = os.path.basename(im_path)
    
    for j,r in enumerate(radius_refinement):

        df = pd.read_csv(os.path.join(results_path_2d, f'radius{r}__{im_dir}__{im_name}.csv'))
        
        if not df.empty:

            for k,s in enumerate(search_range):
            
                beg_time = time.time()

                track = tp.link(df.rename({"z": "frame"}, axis=1), 
                             search_range=s, memory=gap_frames)
                track = tp.filter_stubs(track, threshold=min_frames
                                    ).rename({"frame": "z"}, axis=1)

                # Index of brightest particles
                idx = track.groupby(["particle"])["mass"].transform(max) ==track["mass"]
                df_nms = track[idx]

                # Remove tracks from all spots
                df_without_track = df[
                    ~df.set_index(["x", "y", "z", "mass"]).index.isin(
                        track.set_index(["x", "y", "z", "mass"]).index)]

                # Add back nms (brightest spots)
                df_clean = pd.concat([df_nms, df_without_track]).reset_index(drop=True)
                df_clean.to_csv(os.path.join(results_path_3d ,f'radius{r}__search{s}__{im_dir}__{im_name}.csv'), index=False)

                times[j][k].append(time.time()-beg_time)

        else:
            times[j][k].append(np.nan)

## Test

In [ ]:
test_path = "ufish/data_3d/test/image"

test_ims_paths = glob(os.path.join(test_path, '*.tif'))

In [ ]:
test_results = "ufish/benchmarks/3d/DB/test/results"

In [ ]:
test_results_2d = os.path.join(test_results, "2D")
os.makedirs(test_results_2d, exist_ok=True)

In [ ]:
for i,im_path in enumerate(test_ims_paths):

    im_dir = os.path.basename(os.path.dirname(im_path))
    im_name = os.path.basename(im_path)

    im_3d = tif.imread(im_path)

    r = 2
    
    pad_width = r
        
    df = pd.DataFrame()

    for slice, image_curr in enumerate(im_3d):

        # deepBlink prediction
        yx = pink.inference.predict(image=image_curr, model=model)
        y, x = yx.T.copy()

        # pad to avoid error for spot close to the edges
        yx = yx + pad_width
        image_curr = np.pad(
            image_curr, pad_width=pad_width, mode='constant', constant_values = 0
        )

        # Refinement with trackpy
        df_curr = tp.refine_com(
            raw_image=image_curr, image=image_curr, radius=r, coords=yx
        )
        df_curr["x"] = x
        df_curr["y"] = y
        df_curr["z"] = slice
        df = pd.concat([df, df_curr], ignore_index=True)

    df.to_csv(os.path.join(test_results_2d ,f'radius{r}__{im_dir}__{im_name}.csv'), index=False)

In [ ]:
test_results_3d = os.path.join(test_results, "3D")
os.makedirs(test_results_3d, exist_ok=True)

In [ ]:
r = 2
s = 1

for i,im_path in enumerate(test_ims_paths):
    
    im_dir = os.path.basename(os.path.dirname(im_path))
    im_name = os.path.basename(im_path)
    

    df = pd.read_csv(os.path.join(test_results_2d, f'radius{r}__{im_dir}__{im_name}.csv'))
        
    if not df.empty:

        track = tp.link(df.rename({"z": "frame"}, axis=1), 
                            search_range=s, memory=gap_frames)
        track = tp.filter_stubs(track, threshold=min_frames
                                ).rename({"frame": "z"}, axis=1)

        # Index of brightest particles
        idx = track.groupby(["particle"])["mass"].transform(max) ==track["mass"]
        df_nms = track[idx]

        # Remove tracks from all spots
        df_without_track = df[
            ~df.set_index(["x", "y", "z", "mass"]).index.isin(
                track.set_index(["x", "y", "z", "mass"]).index)]

        # Add back nms (brightest spots)
        df_clean = pd.concat([df_nms, df_without_track]).reset_index(drop=True)
        df_clean.to_csv(os.path.join(test_results_3d ,f'radius{r}_search{s}_{im_dir}_{im_name}.csv'), index=False)
